In [6]:
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,confusion_matrix

In [7]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [9]:
news_df=pd.read_csv('news.csv')

In [10]:
news_df.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [11]:
news_df.tail()

,Unnamed: 0,title,text,label
6330,4490,State Department says it can't find emails fro...,The State Department told the Republican Natio...,REAL
6331,8062,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,FAKE
6332,8622,Anti-Trump Protesters Are Tools of the Oligarc...,Anti-Trump Protesters Are Tools of the Oligar...,FAKE
6333,4021,"In Ethiopia, Obama seeks progress on peace, se...","ADDIS ABABA, Ethiopia —President Obama convene...",REAL
6334,4330,Jeb Bush Is Suddenly Attacking Trump. Here's W...,Jeb Bush Is Suddenly Attacking Trump. Here's W...,REAL


In [12]:
news_df.shape

(6335, 4)

In [13]:
news_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6335 entries, 0 to 6334
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  6335 non-null   int64 
 1   title       6335 non-null   object
 2   text        6335 non-null   object
 3   label       6335 non-null   object
dtypes: int64(1), object(3)
memory usage: 198.1+ KB


In [14]:
news_df.describe(include='O')

,title,text,label
count,6335,6335,6335
unique,6256,6060,2
top,OnPolitics | 's politics blog,"Killing Obama administration rules, dismantlin...",REAL
freq,5,58,3171


In [15]:
news_df.isnull().sum()

,0
Unnamed: 0,0
title,0
text,0
label,0


In [16]:
news_df['content']=news_df['title']+''+news_df['text']

In [17]:
news_df.drop(['Unnamed: 0','text','title'],axis=1,inplace=True)

In [18]:
news_df

,label,content
0,FAKE,"You Can Smell Hillary’s FearDaniel Greenfield,..."
1,FAKE,Watch The Exact Moment Paul Ryan Committed Pol...
2,REAL,Kerry to go to Paris in gesture of sympathyU.S...
3,FAKE,Bernie supporters on Twitter erupt in anger ag...
4,REAL,The Battle of New York: Why This Primary Matte...
...,...,...
6330,REAL,State Department says it can't find emails fro...
6331,FAKE,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...
6332,FAKE,Anti-Trump Protesters Are Tools of the Oligarc...
6333,REAL,"In Ethiopia, Obama seeks progress on peace, se..."


In [19]:
news_df.replace(['FAKE','REAL'],[0,1],inplace=True)

<ipython-input-19-2f06beca8f39>:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  news_df.replace(['FAKE','REAL'],[0,1],inplace=True)


In [20]:
news_df.head()

,label,content
0,0,"You Can Smell Hillary’s FearDaniel Greenfield,..."
1,0,Watch The Exact Moment Paul Ryan Committed Pol...
2,1,Kerry to go to Paris in gesture of sympathyU.S...
3,0,Bernie supporters on Twitter erupt in anger ag...
4,1,The Battle of New York: Why This Primary Matte...


In [21]:
port_stem=PorterStemmer()

In [22]:
def stemming(text):
    stemmed_content=re.sub('[^a-zA:Z]',' ',text)
    stemmed_content=stemmed_content.lower()
    stemmed_content=stemmed_content.split()
    stemmed_content=[port_stem.stem(word) for word in stemmed_content if word not  in stopwords.words('english')]
    stemmed_content=' '.join(stemmed_content)
    return stemmed_content

In [23]:
news_df['content']=news_df['content'].apply(stemming)

In [24]:
X=news_df.drop('label',axis=1)
y=news_df['label']

In [31]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3)

In [32]:
print("Shape of X_train:", X_train.shape)
print("Shape of y_train:", y_train.shape)

Shape of X_train: (4434, 1)
Shape of y_train: (4434,)


In [33]:
vectorizer = TfidfVectorizer()

In [34]:

X_train=vectorizer.fit_transform(X_train['content'])
X_test= vectorizer.transform(X_test['content'])


In [35]:
print("Shape of X_train after TF-IDF:", X_train.shape)
print("Shape of X_test after TF-IDF:", X_test.shape)

Shape of X_train after TF-IDF: (4434, 40327)
Shape of X_test after TF-IDF: (1901, 40327)


In [36]:
lr=LogisticRegression()
lr.fit(X_train,y_train)

LogisticRegression()

In [37]:
lr_pred=lr.predict(X_test)

In [40]:
lr_train=round(lr.score(X_train,y_train)*100,2)
lr_score=round(accuracy_score(y_test,lr_pred)*100,2)
lr_cm=confusion_matrix(y_test,lr_pred)

In [41]:
print('Training Score :',lr_train)
print('Accuracy Score :',lr_score)
print('Confusion Matrix :',lr_cm)

Training Score : 95.62
Accuracy Score : 92.0
Confusion Matrix : [[928  47]
 [105 821]]


In [43]:
rfc=RandomForestClassifier()
rfc.fit(X_train,y_train)

RandomForestClassifier()

In [44]:
rfc_pred=rfc.predict(X_test)

In [45]:
rfc_train=round(lr.score(X_train,y_train)*100,2)
rfc_score=round(accuracy_score(y_test,rfc_pred)*100,2)
rfc_cm=confusion_matrix(y_test,rfc_pred)

In [46]:
print('Training Score :',rfc_train)
print('Accuracy Score :',rfc_score)
print('Confusion Matrix :',rfc_cm)

Training Score : 95.62
Accuracy Score : 90.64
Confusion Matrix : [[878  97]
 [ 81 845]]


In [47]:
model=pd.DataFrame({
    'Model':['Logistic Regression','Random Forest'],
    'Training  Score':[lr_train,rfc_train],
    'Accuracy Score':[lr_score,rfc_score]
})

In [49]:
model.sort_values(by='Accuracy Score').style.background_gradient(
        cmap='coolwarm').set_properties(**{
            'font-family': 'Lucida Calligraphy',
            'font-size': '15px'
        })

,Model,Training Score,Accuracy Score
1,Random Forest,95.620000,90.640000
0,Logistic Regression,95.620000,92.000000


In [50]:
prediction=rfc.predict(X_test)
print(prediction)

[0 0 1 ... 1 1 0]


In [51]:
pred_checking=pd.DataFrame({'Actual':y_test,'Predicted':prediction})
pred_checking.sample(5).style.background_gradient(
        cmap='coolwarm').set_properties(**{
            'font-family': 'Lucida Calligraphy',
            'color': 'LigntGreen',
            'font-size': '15px'
        })

,Actual,Predicted
4297,0,0
707,1,1
3618,1,1
1377,0,0
5817,1,1
